In [238]:
from gurobipy import Model, quicksum, GRB

In [239]:
Plnt = {'plnt'}
DC = set(range(1,3))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = T_Route | D_Route
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [240]:
tp_cost = {(i,j):5 for i,j in TRoute}
del_cost = {(j, k):10 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
dc_run_cost = {(j,ct):1000 for j in DC for ct in CT}
dc_inv_cost = {j:5 for j in DC}

In [241]:
model = Model()

In [242]:
x, y, dc_inv, dc_minus_inv = {}, {}, {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for ct in CT:
        for t in Period:
            y[j,ct,t] = model.addVar(vtype="B", name=f'y[{j},{ct},{t}]')
    
for j in DC:
    for t in Period:
        dc_inv[j,t] = model.addVar(vtype="C", name=f'dc_inv[{j},{t}]')
    
for j in DC:
    dc_minus_inv[j] = model.addVar(vtype="C", name=f'dc_minus_inv[{j}]')

model.update()

In [243]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons = {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            ==
            demand[k,t]
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            demand[k,t] * quicksum(y[j,ct,t] for ct in CT) 
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            + (0 if t==1 else dc_inv[j,t-1] )
            ==
            quicksum(x[j,k,t] for k in Cust)
            + (dc_minus_inv[j] if t==T else dc_inv[j,t])
        )

model.addConstr(dc_inv[1,1] == 30)
    
model.update()

In [244]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(dc_run_cost[j,ct] * y[j,ct,t] for j in DC for ct in CT for t in Period) +
    quicksum(dc_inv_cost[j] * dc_inv[j,t] for j in DC for t in Period) +
    quicksum(dc_inv_cost[j] * dc_minus_inv[j] for j in DC)
    ,GRB.MINIMIZE
)
model.update()

In [245]:
model.optimize()

Optimize a model with 17 rows, 38 columns and 51 nonzeros
Variable types: 30 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 17 rows and 38 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 2850 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.850000000000e+03, best bound 2.850000000000e+03, gap 0.0000%


In [246]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[1,2,2] 10.0
x[2,1,1] 10.0
x[plnt,2,1] 20.0
x[plnt,1,1] 30.0
x[2,2,1] 10.0
x[1,1,2] 10.0
y[1,1,2] 1.0
y[2,1,1] 1.0
dc_inv[1,1] 30.0
dc_minus_inv[1] 10.0
